In [1]:
import sys
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime
from itertools import combinations

VECINOS=40
PREDICCION_REAL = True

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("entrenamiento-listo.csv")
test = pd.read_csv("test-listo.csv")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
entrenamiento = entrenamiento.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])
test = test.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 47 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Region                                   2551 non-null   int64  
 1   Territory                                2551 non-null   int64  
 2   Pricing, Delivery_Terms_Quote_Appr       2551 non-null   int64  
 3   Pricing, Delivery_Terms_Approved         2551 non-null   int64  
 4   Bureaucratic_Code_0_Approval             2551 non-null   int64  
 5   Bureaucratic_Code_0_Approved             2551 non-null   int64  
 6   Submitted_for_Approval                   2551 non-null   int64  
 7   Bureaucratic_Code                        2551 non-null   int64  
 8   Account_Created_Date                     2551 non-null   int64  
 9   Source                                   2551 non-null   int64  
 10  Billing_Country                          2551 no

In [4]:
entrenamiento = entrenamiento.replace([np.inf, -np.inf, np.nan], 0)
test = test.replace([np.inf, -np.inf, np.nan], 0)

entrenamiento_label = (entrenamiento['Stage'] == 'Closed Won').astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage','Fecha'])

if not PREDICCION_REAL:
    test_label  = (test['Stage'] == 'Closed Won').astype(int)
    test = test.drop(columns=['Stage','Fecha'])

scaler = StandardScaler()
scaler.fit(entrenamiento)
set_entrenamiento = scaler.transform(entrenamiento)
set_test = scaler.transform(test)

classifier = KNeighborsClassifier(n_neighbors=VECINOS)
classifier.fit(entrenamiento, entrenamiento_label)
preds = classifier.predict_proba(set_test)

if not PREDICCION_REAL:
    print(confusion_matrix(test_label, preds))
    print(classification_report(test_label, preds))

    classifier.fit(entrenamiento, entrenamiento_label)
    preds = classifier.predict(set_entrenamiento)
    print(confusion_matrix(entrenamiento_label, y_pred))
    print(classification_report(entrenamiento_label, y_pred))


In [5]:
resultados = test[['Opportunity_ID']].copy()
resultados['Target'] = pd.Series([p[1] for p in preds])
#resultados = resultados.groupby('Opportunity_ID').mean()
#resultados = resultados.reset_index()
#resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))

resultados.to_csv("prediccion.csv", index=False)
resultados['Target'].count()

2551